In [1]:
import json
import spacy
from spacy import displacy
import pandas as pd
from tqdm import tqdm

import utils # local file

In [2]:
# carregar características
caracteristicas = utils.carregar_json("../data/caracteristicas.json")
# corrigir caracteristicas duplicadas e letras minusculas
caracteristicas = list(set([c.lower() for c in caracteristicas]))

In [3]:
dados_pessoas = utils.carregar_json("../data/dados_pessoas.json")

In [4]:
nlp_treinado = spacy.load("../data/models/model-best")
nlp_treinado.add_pipe('sentencizer')

nlp_treinado.pipe_names

['tok2vec', 'ner', 'sentencizer']

In [5]:
for pessoa in tqdm(dados_pessoas):
    doc = nlp_treinado(pessoa["biografia"])
    pessoa["caracteristicas"] = []
    # caracteristicas
    for ent in next(doc.sents).ents:
        if (
            ent.label_ == "CARACTERISTICA" # se a entidade for uma caracteristica
            and ent.text.lower() in caracteristicas # se a caracteristica for conhecida
            and ent.text.lower() not in pessoa["caracteristicas"] # se a caracteristica ainda não foi adicionada
        ):
            pessoa["caracteristicas"].append(ent.text.lower()) # adicionar caracteristica
    # sexo
    if pessoa["sexo"]: # se o sexo for conhecido
        pessoa["caracteristicas"].append(pessoa["sexo"].lower()) # adicionar sexo
        if pessoa["sexo"].lower() not in caracteristicas: # se o sexo ainda não foi adicionado
            caracteristicas.append(pessoa["sexo"].lower()) # adicionar sexo
    # ano
    if pessoa["ano"] != None: # se o ano for conhecido
        # se o ano for entre 1900 1950 ou 1950 2000
        if 1900 <= pessoa["ano"] < 1950:
            pessoa["ano"] = 1900
        elif 1950 <= pessoa["ano"] < 2000:
            pessoa["ano"] = 1950

        pessoa["caracteristicas"].append(str(pessoa["ano"])) # adicionar ano
        if str(pessoa["ano"]) not in caracteristicas: # se o ano ainda não foi adicionado
            caracteristicas.append(str(pessoa["ano"])) # adicionar ano


100%|██████████| 1756/1756 [01:03<00:00, 27.64it/s]


In [14]:
# se a pessoa tiver menos de tres caracteristicas, remove
dados_pessoas = [p for p in dados_pessoas if len(p["caracteristicas"]) >= 5]

In [15]:
# save txt
with open("../data/caracteristicas_nlp.txt", "w") as f:
    for p in dados_pessoas:
        f.write(f"{p['nome']}\n")
        for c in p["caracteristicas"]:
            f.write(f"\t{c}\n")
        f.write("\n")

In [16]:
# colunas pessoa + caracteristicas em ordem alfabetica
columns = ["nome"] + sorted(caracteristicas)

df = pd.DataFrame(columns=columns) # dataframe

caracteristicas_nao_tabuladas = []

for pessoa in tqdm(dados_pessoas):
    row = [-1] * len(columns)
    row[0] = pessoa["nome"]
    for caracteristica in pessoa["caracteristicas"]:
        try:
          row[columns.index(caracteristica)] = 1
        except:
          caracteristicas_nao_tabuladas.append(caracteristica)
    df.loc[len(df)] = row


100%|██████████| 824/824 [02:07<00:00,  6.45it/s]


In [17]:
df.to_csv("../data/df.csv", index=False)